# Chapter 2: Two-Point BVPs
---

## Verification and Exploration of Numerical Content

This notebook allows us to verify and explore, with some additional details, the content from Sections 2.2 through 2.4 in the text.

It should be used as a companion, not a replacement, to a careful reading of the text.

## Section 2.2: A Finite Difference Approximation

Starting with (2.14), we have that $A\in\mathbb{R}^{n\times n}$ is given by

$$
\large A = \left(
                \begin{array}{ccccc}
                    2 & -1 & 0 & \cdots & 0 \\
                    -1 & 2 & -1 & \ddots & \vdots \\
                    0 & \ddots & \ddots & \ddots & 0 \\
                    \vdots & \ddots & -1 & 2 & -1 \\
                    0 & \cdots & 0 & -1 & 2
                \end{array}
           \right)
$$

and the data vector is given by

$$
\large b = h^2\left(
                    \begin{array}{c} 
                        f(x_1) \\
                        f(x_2) \\
                        \vdots \\
                        f(x_n)
                    \end{array}
                \right).
$$

- The $h^2$ comes from discretizing the differential operator with the centered finite difference scheme. 

  - It is perhaps slightly strange that we put this on the data vector instead of on $A$.

  - Later, in Section 2.3, we see that $\frac{1}{h^2}A$ defines the difference operator $L_h$. 

- The Gaussian elimination discussed in Section 2.2.3 should be familiar from undergraduate linear algebra.

  - If not, it is straightforward if not a bit messy (you may want to review this here https://en.wikipedia.org/wiki/Gaussian_elimination). 
  
  - The idea is to reduce the system of linear equations into row-echelon form (https://en.wikipedia.org/wiki/Row_echelon_form) and use back-substitution to then determine the vector $v$ such that $Av=b$. 

**Key question:** Since formally we can write $v=A^{-1}b$ (assuming $A^{-1}$ exists), why we do not simply just construct the inverse of the matrix?

**The answer:** Because that is ***expensive*** in terms of FLOPS (https://en.wikipedia.org/wiki/FLOPS) and thanks to Gaussian elimination, completely unnecessary in determining $v$ with less FLOPS. 

**The algorithm:** While you could code up your own version of Gaussian elimination, or even just Algorithm 2.1 in the text (as you have to in Exercise 2.11), there are many available computational libraries that will do this for you efficiently. 
We show how using [`numpy.linalg.solve`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html) below on Example 2.5 from the text.

In [ ]:
import numpy as np
%matplotlib widget 
import matplotlib.pyplot as plt

In [ ]:
# Setup A
n = 5  

def make_A(n):
    A = np.zeros((n,n))
    np.fill_diagonal(A,2)
    A += np.diag(-np.ones(n-1),k=1)
    A += np.diag(-np.ones(n-1),k=-1)
    return A

A = make_A(n)
print(A)

In [ ]:
# Discretize domain and construct data vector 
x = np.linspace(0,1,n+2)
h = x[1]-x[0]
b = h**2 * (3*x+x**2)*np.exp(x)

In [ ]:
v = np.zeros(n+2)
v[1:-1] = np.linalg.solve(A, b[1:-1])  # Numerical soln. using Gaussian elimination

u = x*(1-x)*np.exp(x)  # Exact soln.

%matplotlib widget
plt.figure(1)
plt.plot(x,v,label='Num. Soln. $v$')
plt.plot(x,u,label='Exact Soln. $u$')
plt.legend(loc='upper left', shadow=True)

In [ ]:
# Explore ROC
ns = [5, 10, 20, 40, 80]
E_h = np.zeros(5)
h = np.zeros(5)
alpha_h = np.zeros(5)

count = 0
# Header for table output in format useful for copy/paste into LaTeX
print('  $n$   &   $h$    &     $E_h$    &   Rate of Conv. ' + r'\\ \hline')
for n in ns:
    A = make_A(n)
    
    x = np.linspace(0,1,n+2)
    h[count] = x[1]-x[0]
    b = h[count]**2 * (3*x+x**2)*np.exp(x)

    v = np.zeros(n+2)
    v[1:-1] = np.linalg.solve(A, b[1:-1])  # Numerical soln. using Gaussian elimination

    u = x*(1-x)*np.exp(x)  # Exact soln.
    
    E_h[count] = np.max(np.abs(u-v))
    
    if count == 0:
        print(' {:3d}    &   {:.3f}  &  {:.3e}   &         '.format(n, h[count], E_h[count]) + r'   \\')
    else:
        alpha_h[count] = np.log(E_h[count]/E_h[count-1])/np.log(h[count]/h[count-1])
        print(' {:3d}    &   {:.3f}  &  {:.3e}   &   {:.4f} '.format(n, h[count], E_h[count], alpha_h[count]) + r'  \\')
    
    count +=1

## Section 2.3: Continuous and Discrete Solutions

Let's define a funciton for computing the discrete inner product for functions in $D_h$ like in (2.29).

In [ ]:
def inner_h(u,v,h):
    z = h * (u[0]*v[0] + u[-1]*v[-1])/2.0 + h*np.dot(u[1:-1],v[1:-1])
    return z

### Exercise 2.20:

Prove that if $u,v\in C_0^2((0,1))$ then 
$$
\large    \left| \left< u, v \right> -  \left< u, v \right>_h \right| \leq \frac{h^2}{12} \left|\left| (uv)'' \right|\right|_\infty
$$

In the proof below, we use the well established [error bound](https://en.wikipedia.org/wiki/Trapezoidal_rule#Error_analysis) associated with the [Trapezoidal rule](https://en.wikipedia.org/wiki/Trapezoidal_rule).

***Proof:***

Let $u,v\in C_0^2((0,1))$. 
Then, $uv\in C_0^2((0,1))$.

Let $f=uv$. 
Then, $\int_0^1 f(x)\, dx = \left<u,v\right>$ and $\left<u,v\right>_h$ is identified as the trapezoidal rule applied to this integral.
The result follows immediately from the established error bound for the trapezoidal rule. $\Box$

***Some extra material***

If you've never seen the proof of the error bound for the trapezoidal rule, it follows from applying the integration-by-parts formula choice with some creative choices of integration constants.
I will sketch out the process as a two-step process below that you may find more useful than the Wiki reference linked to above.

(1) Lemma: For $f\in C^2((0,1))$, let $x_i\in[0,1)$ and $h>0$ sufficiently small so that $x_i+h\in[0,1]$

$$
\large \left|\int_{x_i}^{x_i+h} f(x)\, dx - \frac{h}{2}(f(x_i+h)-f(x_i)) \right| \leq \frac{h^3}{12}\left|\left|  f''\right|\right|_\infty.
$$

Sketch of Proof: A simple change of variables (to simplify the limits of integration) followed by integrating-by-parts twice with clever choices of integration constants leads to

$$
 \large    \int_{x_i}^{x_i+h} f(x)\, dx = \int_0^h f(t+x_i)\, dt = \underbrace{\frac{h}{2}\left[f(x_i) + f(x_i+h)\right]}_{\text{Trap. Rule}} + \underbrace{\int_{0}^{h} \left(\frac{(t-h/2)^2}{2}-\frac{h^2}{8} \right)f''(t+x_i)\, dt}_{\text{Error in Trap. Rule on $(x_i, x_i+h)$}}.
$$

So it follows that the error can be bounded by

$$
 \large \left| \int_{0}^{h} \left(\frac{(t-h/2)^2}{2}-\frac{h^2}{8} \right)f''(t+x_i)\, dt \right| \leq \left|\left|f''\right|\right|_\infty \int_0^h \left| \frac{(t-h/2)^2}{2}-\frac{h^2}{8} \right| \, dt. 
$$

The integrand is the absolute value of a parabola $\displaystyle \frac{(t-h/2)^2}{2}-\frac{h^2}{8}$ which opens upward and is zero whenever $t-h/2 = \pm h/2$ (i.e., at $t=0$ and at $t=h$), so for $t\in(0,h)$, we have that

$$
   \large \left|\frac{(t-h/2)^2}{2}-\frac{h^2}{8}\right| = \frac{h^2}{8} - \frac{(t-h/2)^2}{2}. 
$$

It follows from a direct integration of this term that the error in the trapezoidal rule on $(x_i,x_i+h)$ is bounded by

$$
 \large \left| \int_{0}^{h} \left(\frac{(t-h/2)^2}{2}-\frac{h^2}{8} \right)f''(t+x_i)\, dt \right| \leq \frac{h^3}{12}\left|\left|f''\right|\right|_\infty.
$$

(2) Since we apply the trapezoidal rule on $n$ subintervals, we add up the error bound $n$ times and use the fact that $h=1/n$ to get the result used in the proof of Exercise 2.20 above. $\Box$

### Lemma 2.3 and 2.4: Symmetric Positive Defeniteness of the operator $L_h$

The book provides one way of proving this. 
Here's another.

For $u\in D_{h,0}$, we observe that $L_h u$ is simply given by $\frac{1}{h^2} A\tilde{u}$ where $\tilde{u}$ is the vector defined by evaluating $u$ at $x_i$ for $i=1,\ldots, n$ and the matrix $A$ is as given above.
Moreover, for $u,v\in D_{h,0}$ we also have that $\left<u,v\right>_h$ simplifies to $h(\tilde{u},\tilde{v})$ where $(\cdot,\cdot)$ denotes the usual Euclidean inner product.
Now, with this notation, and the fact that $A$ is a SPD matrix, the SPD of the operator $L_h$ follows immediately. 

### A maximum principle for the discrete problem

One take home message is that the discrete analog of the Dirac delta function is given by $\frac{1}{h} e^k$ where $e^k\in D_{h,0}$ is defined by $e^k(x_k)=1$ and $e^k(x_j)=0$ if $j\neq k$.
Implicit is the assumption that we also require that $1\leq k\leq n$, i.e., that we evaluate at an interior point since $e^k\in D_{h,0}$ automatically implies that $e^k(x_0)=0=e^k(x_{n+1})$).

Another take home message is that the discrete Green's function is given by $G^k(x_j) = G(x_j,x_k)$ where $G$ is the Green's function for the continuous problem so that $L_h G^k = \frac{1}{h}e^k$. 

Let's explore this a bit numerically.

In [ ]:
def G(x,y): 
    if 0 <= y <= x:
        z = y*(1-x)
    else:
        z = x*(1-y)
    return z

def make_G(n, x):    
    G_k = np.zeros((n+2,n+2))
    for j in range(0,n+2):
        for k in range(0,n+2):
            G_k[j,k] = G(x[j],x[k])
    return G_k

In [ ]:
n = 5

x = np.linspace(0,1,n+2)
h = x[1]-x[0]  # So 1/h = n+1

G_k = make_G(n,x)
        
A = make_A(n)

In [ ]:
# Choose an integer between 0 and n+1
k = 2

test = np.zeros(n+2)
test[1:-1] = 1/h**2 * np.dot(A,G_k[1:-1,k])

%matplotlib widget
plt.figure(2) 
plt.plot(x,test)  

With the inner product notation, we have for the continuous problem $u(x) = \left< G(x,y),f(y) \right>$ (where the integral is with respect to $y$ not $x$), and now for the discrete problem we also have that $v(x_j) = \left< G^k(x_j), f \right>_h$ (where the summation is with repect to $k$ not $j$).

This implies another way for construction solutions using (2.33), which we explore numerically.

In [ ]:
# Another way to construct solutions using (2.33)

b_old = h**2 * (3*x+x**2)*np.exp(x)

v_old = np.zeros(n+2)
v_old[1:-1] = np.linalg.solve(A, b_old[1:-1])  # Numerical soln. using Gaussian elimination

b_new = (3*x+x**2)*np.exp(x)

v_new = np.zeros(n+2)
for j in range(1,n+1):
    v_new[j] += inner_h(G_k[j,:],b_new,h)

u = x*(1-x)*np.exp(x)  # Exact soln.

%matplotlib widget
plt.figure(3)
plt.plot(x, v_old, 'b.', markersize=15, label='Num. Soln. $v_{old}$')
plt.plot(x, v_new, 'g--', label='Num. Soln. $v_{new}$')
plt.legend(loc='upper left', shadow=True)

Let's make sure Proposition 2.6 holds for our numerically constructed discrete Green's functions.

In [ ]:
ns = range(1,30)

for n in ns:
    x = np.linspace(0,1,n+2)
    h = x[1]-x[0]  # So 1/h = n+1

    G_k = make_G(n, x)
    
    temp = np.zeros(n+2)
    for i in range(0,n+2):
        temp[i] = inner_h(G_k[:,i], np.ones(n+2), h)
        
    print(np.max(temp))

### A word of caution about discrete Green's functions.

If we know $G$ for the continuous problem, then it appears as if determining the discrete Green's function is rather trivial and we can use it to easily determine solutions through simple computations of inner products.

Formally, we think of $G$ as the inverse of the differential operator $L$ applied to $\delta(x-y)$, and we can think of the discrete Green's function in a similar way as the inverse of $L_h$ applied to $e^k$. This simply means that the discrete Green's funciton may be constructed by determining $\frac{1}{h^2}A^{-1}$ and then multipling this to the standard basis vectors (scaled by $1/h$) of $\mathbb{R}^n$. 

However, this is a ***stupid*** computation because it involves inverting a matrix, which is really expensive and something we try to avoid whenever possible. 

Just like with regular Green's functions, which are really difficult to determine in general cases, the mere existence of one is usually enough to infer useful properties of the solution similar to how the existence of the inverse of a matrix is also useful even if we never construct it.

## Section 2.4: Eigenvalue Problems

Let's verify Lemma 2.9 numerically. First, we need to ***correct*** it since the book version contains an error.

<hr style="border:5px solid cyan"> </hr>

<span style="background-color:rgba(0, 255, 255, 0.5)">Corrected Lemma:</span>

The difference operator $L_h$ has eigenvalues

$$
    \mu_k = \frac{4}{h^2}\sin^2(k\pi h/2), \qquad k=1,\ldots, n.
$$

The corresponding eigenfunctions $v_k\in D_{h,0}$ are given by

$$
    v_k = \left(
                \begin{array}{c} 
                        \sin(k\pi x_1) \\
                        \sin(k\pi x_2) \\
                        \vdots \\
                        \sin(k\pi x_n)
                 \end{array}
           \right).
$$

Furthermore, the discrete eigenfunctions are orthogonal and satisfy

$$\large
    \langle v_k, v_m \rangle = \begin{cases}
                                    0, & k\neq m, \\
                                    \frac{1}{2 \color{red}{h}}, & k=m.
                               \end{cases}
$$

<hr style="border:5px solid cyan"> </hr>

Before we look at plots of the numerically estimated eigenfunctions and their errors due to numerical approximation, we need to understand a few key points. 

<span style="background-color:rgba(255, 0, 255, 0.5)">***Key points:***</span>

- Using a linear algebra package to estimate the eigenvalues/eigenfunctions of a matrix will typically produce results that are normalized with respect to the Euclidean norm (i.e., the 2-norm).

- This implies that the $v_k$ computed from a linear algebra package will likely have the property that $\langle v_k, v_k \rangle = 1$.

- The sign of an eigenvector is completely arbitrary, and we could have just as easily stated that the discrete eigenfunctions were computed using $-\sin$ in place of $\sin$ in the lemma above.

In [ ]:
n = 30
x = np.linspace(0,1,n+2)
h = x[1]-x[0]

A = make_A(n)

mu, v = np.linalg.eigh(1/h**2 * A)  # eigh returns eigs of symmetric matrix in increasing order

# Lemma 2.9
mu_exact = np.zeros(n)
for k in range(0,n):
    mu_exact[k] = 4/(h**2) * (np.sin((k+1)*np.pi*h/2))**2

In [ ]:
print(mu)
print('-'*50) 
print(np.max(np.abs(mu-mu_exact)))  # Largest error made in estimating an eigenvalue

In [ ]:
v_exact = np.zeros((n,n))
for k in range(0,n):
    v_exact[:,k] = np.sin((k+1)*np.pi*x[1:-1])

In [ ]:
# Note that the eig functions in numpy will normalize eigenvectors 
# with respect to the usual 2-norm and the sign is arbitrary

# Choose k between 1 and 30
k = 3

print('Component #' + str(k) + ' should be 1 and the others\n' +\
      'should be approximately zero for estimated eigenvectors')
print('-'*50)
print(v.T.dot(v)[k-1, :])

print('\n\n')
print('-'*50)
print('Component #' + str(k) + ' should be ' + str(1/(2*h)) + ' and the others\n' +\
      'should be approximately zero for eigenvectors from Lemma')
print(v_exact.T.dot(v_exact)[k-1,:])

In [ ]:
%matplotlib widget
# Pick k between 1 and 30
k = 3

ind = np.argmax(np.abs(v[:,k-1]))

plt.figure(4)
plt.plot(x[1:-1],v[:,k-1]/v[ind,k-1])
plt.title('$L^\infty$ normalized estimated numerical e.func #' + str(k))

plt.figure(5)
plt.plot(x[1:-1],v[:,k-1]/v[ind,k-1]-np.sin(k*np.pi*x[1:-1]))
plt.title('Difference from exact to estimated numerical e.func #' + str(k))

In [ ]:
def g_spectral(x, g, v, n, trunc, norm_const=1):
    
    g_sum = np.zeros(n+2)
    
    for k in range(trunc):
        g_sum[1:-1] += np.dot(g[1:-1],v[:,k])*v[:,k] / norm_const
    
    return g_sum

In [ ]:
g = x*(1-x)*np.exp(x)

In [ ]:
%matplotlib widget
plt.figure(6)
plt.plot(x, g)
plt.plot(x, g_spectral(x, g, v, n, trunc=2))

In [ ]:
%matplotlib widget
plt.figure(7)
plt.plot(x, g)
plt.plot(x, g_spectral(x, g, v_exact, n, trunc=2, norm_const = v_exact[:,0].dot(v_exact[:,0])))
plt.title('Normalizing constant used = ' + str(v_exact[:,0].dot(v_exact[:,0])))